In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
df_raw = pd.read_csv("CoST.csv")
# df = pd.read_csv("https://data.4tu.nl/articles/dataset/Corpus_of_Social_Touch_CoST_/12696869?file=24044075", sep='\t')

# Preprocessing

frame = 135 - 1 second

In [2]:
df = df_raw.copy()
df.columns = df.columns.str.strip(" ")
df = df.set_index(['subject', 'variant', 'gesture'])
df['frame'].value_counts()

1       7805
7       7805
10      7805
9       7805
8       7805
        ... 
1580       1
1581       1
1582       1
1583       1
1747       1
Name: frame, Length: 1747, dtype: int64

In [3]:
df['observation'] = np.nan
df = df.reset_index()
values = df['frame'].values
i = 1
for index, element in tqdm(enumerate(values)):
    if values[index + 1] < values[index]:
        df.loc[index, "observation"] = i
        i += 1

0it [00:00, ?it/s]

IndexError: index 1496855 is out of bounds for axis 0 with size 1496855

In [5]:
df['observation'] = df['observation'].fillna(method='bfill')
df['observation'] = df['observation'].fillna(df['observation'].max() + 1)
df['observation'] = df['observation'].astype(int)

In [6]:
# Create ML dataset

data = pd.DataFrame(index=np.arange(1, df['observation'].max() + 1).astype(int), columns=['duration'])

# Add gesture (y)
data['gesture'] = pd.Series(df.drop_duplicates(['observation'], keep='last')['gesture'].values)

dict_gesture = {1: "grab", 2: "hit", 3: "massage", 4: "pat", 5: "pinch",
                6: "poke", 7: "press", 8: "rub", 9: "scratch", 10: "slap", 11: "squeeze",
                12: "stroke", 13: "tap", 14: "tickle"}

data['gesture'] = data['gesture'].map(dict_gesture)

data['variant'] = pd.Series(df.drop_duplicates(['observation'], keep='last')['variant'].values)
dict_variant = {1: "gentle", 2: "normal", 3: "rough"}
data['variant'] = data['variant'].map(dict_variant)


# Add duration for every observation
data['duration'] = (df.drop_duplicates(['observation'], keep='last')['frame'] / 135).values

Some unique combination of subject, variant, gesture have 5 repetitions. 7805 gesture captures

# Feature engineering

– Mean pressure is the mean over channels and time (1).

– Maximum pressure is the maximum value over channels
and time (2).

– Pressure variability is the mean over time of the sum over
channels of the absolute value of difference between two
consecutive frames (3).

– Mean pressure per row is the mean over columns and time
resulting in one feature per row which are in the direction
of the mannequin arm’s length (from top to bottom, 4–
11).

– Mean pressure per column is the mean over rows and
time resulting in one feature per column which are in
the direction of the mannequin arm’s width (from left to
right, 12–19).

– Contact area per frame is the fraction of channels with a
value above 50 % of the maximum value. Mean contact
area is the mean over time of contact area (20) and the
maximum pressure contact area is the contact area of the
frame with the highest mean pressure over channels (21).
The size of the contact area indicated whether the whole
hand was used for a touch gesture, as would be expected

In [7]:
# Mean pressure
ch_cols = [i for i in df.columns if i.startswith("ch")]
data['mean_pressure'] = df.groupby('observation')[ch_cols].mean().mean(axis=1)

In [189]:
data.groupby(['gesture'])['mean_pressure'].mean()

gesture
grab       341.931508
hit        127.462967
massage    207.492889
pat        128.959427
pinch      148.889580
poke       118.970094
press      201.823134
rub        164.400707
scratch    138.925870
slap       120.755853
squeeze    285.747048
stroke     149.573673
tap        117.651540
tickle     127.242386
Name: mean_pressure, dtype: float64

In [8]:
# Maximum pressure
data['maximum_pressure'] = df.groupby('observation')[ch_cols].max().max(axis=1)

In [211]:
data.groupby(['gesture'])['maximum_pressure'].mean() * 2

gesture
grab       1708.850987
hit        1689.304659
massage    1693.558348
pat        1513.917415
pinch      1691.086022
poke       1534.254480
press      1703.835125
rub        1595.583483
scratch    1539.315412
slap       1579.863799
squeeze    1756.057451
stroke     1539.482014
tap        1491.870968
tickle     1444.904847
Name: maximum_pressure, dtype: float64

In [9]:
# Variance over channels and time (44)
data['variance'] = df.groupby('observation')[ch_cols].var().var(axis=1)

In [12]:
# Contact area per frame
# Attention! It's really time and memory expensive
df['contact_area'] = df[ch_cols].apply(lambda x: np.mean(x > x.max() * 0.5), axis=1)
data['mean_contact_area'] = df.groupby('observation')['contact_area'].mean()

In [62]:
data

,duration,gesture,variant,mean_pressure,maximum_pressure,variance,mean_contact_area
1,0.474074,grab,gentle,136.772461,439,1.346562e+07,0.309570
2,0.496296,grab,gentle,193.445196,594,1.043202e+08,0.410215
3,0.496296,grab,gentle,112.382929,515,1.193715e+07,0.233442
4,0.651852,grab,gentle,156.530717,657,1.455307e+08,0.208452
5,0.896296,grab,gentle,156.828771,786,6.985941e+08,0.309788
...,...,...,...,...,...,...,...
7801,2.896296,tickle,rough,171.512228,853,8.934892e+07,0.210358
7802,4.029630,tickle,rough,141.360754,864,3.718105e+07,0.226218
7803,3.207407,tickle,rough,162.377057,885,1.146045e+08,0.205362
7804,2.962963,tickle,rough,149.128516,890,1.321851e+08,0.176523


In [59]:
df

,subject,variant,gesture,frame,ch1,ch2,ch3,ch4,ch5,ch6,...,ch59,ch60,ch61,ch62,ch63,ch64,observation,channel_max,index,contact_area
0,1,1,1,1,22,83,55,68,69,66,...,31,36,35,35,28,22,1,146,0,0.296875
1,1,1,1,2,21,83,56,68,68,65,...,33,40,38,38,28,21,1,146,1,0.312500
2,1,1,1,3,21,80,55,69,70,65,...,30,37,36,36,26,23,1,146,2,0.312500
3,1,1,1,4,21,81,53,65,69,65,...,32,35,36,34,28,21,1,128,3,0.406250
4,1,1,1,5,13,77,54,75,68,56,...,31,36,32,33,29,22,1,132,4,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496850,31,3,14,411,103,68,43,67,76,86,...,14,19,20,20,14,20,7805,150,1496850,0.343750
1496851,31,3,14,412,108,72,43,73,78,90,...,16,27,25,27,15,18,7805,159,1496851,0.343750
1496852,31,3,14,413,107,71,46,79,78,92,...,17,24,27,29,14,19,7805,167,1496852,0.343750
1496853,31,3,14,414,107,71,46,75,81,92,...,20,27,28,27,19,15,7805,166,1496853,0.343750
